В цьому домашньому завданні ми проведемо додаткові експерименти для рішення задачі бінарної класифікації і створимо ваш новий submission на змагання на Kaggle.

-----------


**Завдання 0**. Завантажте дані `train.csv`, `test.csv`, `sample_submission.csv` зі змагання на Kaggle - шукайте посилання в уроці [Запрошення до участі у Kaggle-змаганні.](https://data-loves.kwiga.com/courses/machine-learning-dlia-liudei/domashnie-zavdannia-zmagannia-z-kaggle)  Для завантаження потрібно долучитись до змагання (натиснути кнопку "Join").


**Завдання 1**. **Збираємо весь код з попереднього ДЗ в одному місці.** В лекційному ноутбуці `Логістична регресія з ScikitLearn. Повна ML задача.ipynb` ми познайомились з поняттям пайплайнів, а також я показала, як компактно виглядає рішення МЛ задачі, якщо ми зберемо весь код разом.

Оскільки ми далі будемо робити експерименти, які включають ті самі етапи попередньої обробки, але інше моделювання - буде зручно мати весь код компактно і під рукою. Тому зараз ми займемось збором коду до купи :) Після цього завдання для подальших експериментів ви можете перенести частини розвʼязку взагалі в окремий `.py` файл, аби було зручно імпортувати функції.

Зі свого рішення в попередньому домашньому завданні (`Логістична регресія з scikit learn.ipynb`) зберіть усі кроки розвʼязку задачі разом з використанням `sklearn.Pipeline` за прикладом з лекції.

Ваш код нижче має містити
1. Читання даних з файлу (поза пайплайном).
2. Розбиття на тренувальний і валідаційний набори, де валідаційний містить 20% даних (поза пайплайном).
3. Виділення категоріальних і числових колонок (поза пайплайном).
4. Підготовку категоріальних і числових колонок (частина пайплайну). В прикладі в лекції ми оформлювали обробку числових і категоріальних колонок в окремі трансформери `numeric_transformer`, `categorical_cols`. Рекоемндую зробити саме так, так потім зручніше вносити зміни :)
5. Тренування лог регресії (частина пайплайну).
6. Запуск пайплайну на тренування на трен. даних (поза пайплайном).
7. Запуск пайплайну на передбачення на трен і вал. даних і вимір метрик якості ROC-AUC + вивдення Confusion Matrix (поза пайплайном).
8. Збереження моделі в формат joblib (поза пайплайном).

Ви це все вже зробили в попереднтьому ДЗ! Тож, тут просто заадча все зібрати разом.

Нижче я додала підказки, що покроково ви маєте зробити. Якщо ви почуваєтесь впевнено, можете видалити ці підказки і реалізувати все самостійно, або ж - просто заповнити пропуски.

Завдання оцінюється в 10 балів. Головний результат - аби код в фіналі був робочий. Бо за не робочий нам гроші ніхто не заплатить :)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
raw_df = pd.read_csv("drive/MyDrive/Machine Learning для людей/data/bank/train.csv")

train_df, val_df = train_test_split(raw_df, test_size=0.2, random_state=42, stratify=raw_df['Exited'])

target_col = 'Exited'
input_cols = list(train_df.columns.drop([target_col, 'id', 'CustomerId', 'Surname']))
numeric_cols = train_df[input_cols].select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_df[input_cols].select_dtypes(include='object').columns.tolist()

X_train = train_df[input_cols]
y_train = train_df[target_col]
X_val = val_df[input_cols]
y_val = val_df[target_col]

numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42))
])

model_pipeline.fit(X_train, y_train)

def evaluate_pipeline(pipeline, X, y, name):
    preds = pipeline.predict(X)
    probs = pipeline.predict_proba(X)[:, 1]

    auc = roc_auc_score(y, probs)
    f1 = f1_score(y, preds)
    conf_mat = confusion_matrix(y, preds)

    print(f"\n--- {name} Metrics ---")
    print(f"AUROC: {auc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_mat)
    return probs

train_probs = evaluate_pipeline(model_pipeline, X_train, y_train, "Train")
val_probs = evaluate_pipeline(model_pipeline, X_val, y_val, "Validation")

joblib.dump(model_pipeline, 'log_reg_pipeline.joblib')
print("\nПайплайн збережено в 'log_reg_pipeline.joblib'")


--- Train Metrics ---
AUROC: 0.8824
F1 Score: 0.6352
Confusion Matrix:
[[9177  381]
 [1128 1314]]

--- Validation Metrics ---
AUROC: 0.8797
F1 Score: 0.6412
Confusion Matrix:
[[2271  119]
 [ 266  344]]

Пайплайн збережено в 'log_reg_pipeline.joblib'


**Завдання 2**. Такс, у нас з вами є вже готовий пайплайн. Давайте проведемо нові експерименти.

  Додайте в попередню обробку числових колонок генерацію polinomal features до степені 2 включно. Для цього створіть новий препроцесор і створіть новий пайплайн.

  Запустіть пайплайн на тренування і виведіть метрики для тренувального і валідаційного набору. Напишіть, як вам модель? Чи спостерігається в цій моделі overfit чи underfit? Чи ця модель добре генералізує?

In [20]:
from sklearn.preprocessing import PolynomialFeatures

numeric_transformer_poly = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', MinMaxScaler())
])

preprocessor_poly = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_poly, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

poly_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_poly),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42))
])

print("Навчання моделі з поліноміальними ознаками...")
poly_pipeline.fit(X_train, y_train)

print("\nРезультати з поліноміальними ознаками:")
evaluate_pipeline(poly_pipeline, X_train, y_train, "Train Poly")
evaluate_pipeline(poly_pipeline, X_val, y_val, "Validation Poly")

Навчання моделі з поліноміальними ознаками...

Результати з поліноміальними ознаками:

--- Train Poly Metrics ---
AUROC: 0.9110
F1 Score: 0.6754
Confusion Matrix:
[[9221  337]
 [1025 1417]]

--- Validation Poly Metrics ---
AUROC: 0.9099
F1 Score: 0.6771
Confusion Matrix:
[[2283  107]
 [ 243  367]]


array([0.5836549 , 0.03540179, 0.06650954, ..., 0.01911462, 0.04267899,
       0.0678362 ])

**Завдання 3**. Тепер давайте створимо ще новий пайплайн, тільки тепер поліноміальні ознаки згенеруємо до степені 4. Зробіть висновок про якість моделі. Якщо вам подобається резульат якоїсь з моделей в цьому ДЗ - рекомендую зробити submission в змаганні.

In [21]:
numeric_transformer_poly4 = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=4, include_bias=False)),
    ('scaler', MinMaxScaler())
])

preprocessor_poly4 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_poly4, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

poly4_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_poly4),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42, max_iter=1000))
])

print("⏳ Навчання моделі")
poly4_pipeline.fit(X_train, y_train)

print("\n--- Результати (Degree 4) ---")
evaluate_pipeline(poly4_pipeline, X_train, y_train, "Train Poly 4")
evaluate_pipeline(poly4_pipeline, X_val, y_val, "Validation Poly 4")

test_raw_df = pd.read_csv("drive/MyDrive/Machine Learning для людей/data/bank/test.csv")

test_probs_poly4 = poly4_pipeline.predict_proba(test_raw_df)[:, 1]

submission_poly4 = pd.DataFrame({
    'id': test_raw_df['id'],
    'Exited': test_probs_poly4
})
submission_poly4.to_csv('submission_poly_4.csv', index=False)
print("\n Файл 'submission_poly_4.csv' створено.")

⏳ Навчання моделі

--- Результати (Degree 4) ---

--- Train Poly 4 Metrics ---
AUROC: 0.9317
F1 Score: 0.7286
Confusion Matrix:
[[9203  355]
 [ 839 1603]]

--- Validation Poly 4 Metrics ---
AUROC: 0.9283
F1 Score: 0.7191
Confusion Matrix:
[[2284  106]
 [ 208  402]]

 Файл 'submission_poly_4.csv' створено.


**Завдання 4. Перенавчання і регуляризація**.

  Скачайте набір даних `regression_data.csv`. Звичайте набір даних з `regression_data.csv`, розбийте на train і test (в тест 20%) і натренуйте модель лінійної регресії з масштабуванням числових ознак і поліноміальними ознаками до степені **5 включно**.

  Виміряйте якість прогностичної моделі і зробіть висновок, чи модель хороша, чи вона добре генералізує?


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

df_reg = pd.read_csv("drive/MyDrive/Machine Learning для людей/data/regression_data.csv")

X = df_reg.drop(columns=['target'])
y = df_reg['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=5, include_bias=False)),
    ('scaler', MinMaxScaler()),
    ('model', LinearRegression())
])

reg_pipeline.fit(X_train, y_train)

def evaluate_regression(pipeline, X, y, name):
    preds = pipeline.predict(X)
    mse = mean_squared_error(y, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y, preds)
    print(f"--- {name} Metrics ---")
    print(f"RMSE (Середньоквадратична помилка): {rmse:.4f}")
    print(f"R2 Score (Точність): {r2:.4f}")
    return r2

print("Результати моделі з поліномом 5-го ступеня:")
r2_train = evaluate_regression(reg_pipeline, X_train, y_train, "TRAIN")
r2_test = evaluate_regression(reg_pipeline, X_test, y_test, "TEST")

Результати моделі з поліномом 5-го ступеня:
--- TRAIN Metrics ---
RMSE (Середньоквадратична помилка): 0.0000
R2 Score (Точність): 1.0000
--- TEST Metrics ---
RMSE (Середньоквадратична помилка): 12.3633
R2 Score (Точність): 0.9345


на тренуванні помилка нульова, а на тесті з'являється значна похибка, свідчить про перенавчання. Модель із поліномом 5-го ступеня стала занадто "гнучкою". Замість того щоб намалювати плавну лінію тренду, вона почала вигинатися, щоб пройти через кожну, навіть випадкову, точку тренувального набору. Через це на нових даних вона помиляється більше, ніж могла б простіша модель.

---



**Завдання 5**. Натренуйте моделі Lasso(), Ridge(), ElasaticNet() на цих даних (з поліном ознаками до степені 20 включно), порівняйте якість з тою, яка була отримана з лінійною регресією. Яка модель найкраще генералізує і чому на ваш погляд (можливо треба буде для відповіді зробити додатковий аналіз ознак)?

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

try:
    df_reg = pd.read_csv("drive/MyDrive/Machine Learning для людей/data/regression_data.csv")
except FileNotFoundError:
    print("⚠️ Файл не знайдено!")

X = df_reg.drop(columns=['target'])
y = df_reg['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def get_pipeline(model):
    return Pipeline([
        ('poly', PolynomialFeatures(degree=20, include_bias=False)),
        ('scaler', MinMaxScaler()),
        ('model', model)
    ])

models = [
    ('LinearRegression', LinearRegression()),
    ('Lasso', Lasso(alpha=0.1, max_iter=10000, random_state=42)),
    ('Ridge', Ridge(alpha=1.0, random_state=42)),
    ('ElasticNet', ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000, random_state=42))
]

print(f"{'Model':<20} | {'Train R2':<10} | {'Test R2':<10} | {'Test RMSE':<10}")
print("-" * 65)

for name, model in models:
    pipeline = get_pipeline(model)
    pipeline.fit(X_train, y_train)

    train_pred = pipeline.predict(X_train)
    test_pred = pipeline.predict(X_test)

    r2_train = r2_score(y_train, train_pred)
    r2_test = r2_score(y_test, test_pred)
    rmse_test = np.sqrt(mean_squared_error(y_test, test_pred))

    print(f"{name:<20} | {r2_train:<10.4f} | {r2_test:<10.4f} | {rmse_test:<10.4f}")

print("\n--- Lasso ---")
lasso_pipeline = get_pipeline(Lasso(alpha=0.1, max_iter=10000, random_state=42))
lasso_pipeline.fit(X_train, y_train)
lasso_model = lasso_pipeline.named_steps['model']

total_features = len(lasso_model.coef_)
non_zero_features = np.sum(np.abs(lasso_model.coef_) > 1e-4)

print(f"Всього створено ознак (поліном 20): {total_features}")
print(f"Lasso залишила важливих ознак: {non_zero_features}")
print(f"Lasso викинула (занулила) сміття: {total_features - non_zero_features}")

Model                | Train R2   | Test R2    | Test RMSE 
-----------------------------------------------------------------
LinearRegression     | 1.0000     | 0.8947     | 15.6743   
Lasso                | 0.9993     | 0.9996     | 1.0046    
Ridge                | 0.9620     | -1.5125    | 76.5627   
ElasticNet           | 0.8184     | -5.6896    | 124.9292  

--- Lasso ---
Всього створено ознак (поліном 20): 53129
Lasso залишила важливих ознак: 1
Lasso викинула (занулила) сміття: 53128


Найкраще генералізує модель Lasso. Вона показала майже ідеальний результат на тестових даних ($R^2 = 0.9996$) і найменшу помилку ($RMSE \approx 1.0$), при цьому результат на тренуванні та тесті майже однаковий, що свідчить про повну відсутність перенавчання.